In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
!ls CDATA

AUS.csv  FIN.csv  FRA.csv  GBR.csv


In [3]:
ccodes = ['AUS', 'FRA', 'GBR']
datas = ['./CDATA/{}.csv'.format(x) for x in ccodes]

In [4]:
datas[0]

'./CDATA/AUS.csv'

In [5]:
datas

['./CDATA/AUS.csv', './CDATA/FRA.csv', './CDATA/GBR.csv']

In [6]:
data = pd.read_csv('./CDATA/AUS.csv', index_col='Unnamed: 0')

In [7]:
data

,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
90399,9.090001,10.126582,9.691745,11.145511,10.113563,3.950185,6.739049,9.084532,8.488746,7.231772,...,3.538487,2.332362,4.352643,1.820112,2.845226,3.303850,1.762780,2.449889,2.487923,1.508367
91023,0.455171,-0.030040,2.108038,1.404348,2.052476,3.365892,7.449047,8.087802,7.492809,6.410574,...,2.425093,3.067561,4.182300,1.044141,6.206931,1.462691,4.820785,6.356251,4.447267,6.321014
91252,6.300000,6.100000,5.800000,7.200000,10.000000,9.000000,8.300000,8.100000,8.100000,7.200000,...,4.800000,4.400000,4.200000,5.600000,5.200000,5.100000,5.200000,5.700000,6.100000,6.100000


In [8]:
datas = [pd.read_csv(x, index_col='Unnamed: 0').iloc[:,:-1] for x in datas]
for i, data in enumerate(datas):
    data.columns = data.columns.astype('int')
    data = data.astype(floatX)
    
    new_data = np.concatenate([data.values.T[1:], data.values.T[:-1]], axis=1)
    new_data_columns = data.columns[1:]
    new_data = pd.DataFrame(new_data.T, columns=new_data_columns)
    data = new_data
    datas[i] = data
    print(data.columns)

Int64Index([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
            1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
            2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
            2013, 2014],
           dtype='int64')
Int64Index([1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
            1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
            1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002],
           dtype='int64')
Int64Index([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
            2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012],
           dtype='int64')


In [9]:
[data.shape for data in datas]

[(6, 35), (6, 32), (6, 23)]

In [10]:
datas[0].values.shape

(6, 35)

In [11]:
country_data = {c:d for c,d in zip(ccodes, datas)}

In [12]:
class VARmodel:
    def __init__(self, data, name='VARmodel', mu=None, current_year=None):
        self.data_raw = data
        self.years = data.columns.values.astype('int32')
        years_c = tf.constant(data.columns.values, dtype=tf.int32, name='data_years')

        if current_year is None:
            self.OBSERV_STEPS = np.Infinity
        else:
            self.OBSERV_STEPS = tf.reduce_sum(tf.cast(years_c <= current_year, tf.int32))
        
        self.NUM_STEPS = data.shape[1]
        self.name = name
        self.logdensities = []
        self.priors = []
        self.dim = [3,3*2+1]
        self.summaries = []
        
        self.observable_mask = tf.range(0, self.NUM_STEPS, dtype=tf.int32) < self.OBSERV_STEPS
        
        pd = np.mean(np.std(data.values[:,1:] - data.values[:,:-1], axis=-1))
        
        with tf.variable_scope(name) as scope:
            self.data = tf.get_variable(initializer=data.values.T[np.newaxis], 
                                    trainable=False, name='data')
            self.scope = scope
            
            self.create_rw_priors()
            self.outputs = self.create_walk_inference(mu=mu)
            self.create_observ_dispersion_inference(pd*0.5)
            self.create_likelihood(self.observable_mask, self.outputs)
            self.summary = tf.summary.merge(self.summaries)
            
    def create_summary(self, stype, name, tensor):
        s = stype(name, tensor)
        self.summaries.append(s)

    def create_rw_priors(self):
        dim = self.dim
        with tf.variable_scope('rw_priors'):
            with tf.variable_scope('walk_ord'):
                s1_prior_d = LogNormal(1, mu=math.log(0.01), sigma=6., name='s1_prior')

                with tf.variable_scope('s1_inference', dtype=floatX):
                    mu = tf.get_variable('mu', shape=[1], 
                                         initializer=tf.constant_initializer(s1_prior_d.mu))

                    logsigma_init = tf.constant_initializer(min(math.log(s1_prior_d.sigma), -1.))
                    logsigma = tf.get_variable('logsigma', shape=[1], 
                                               initializer=logsigma_init)
                    sigma = tf.exp(logsigma)
                    s1_d = LogNormal(1, mu=mu, sigma=sigma)

                s1 = s1_d.sample()
                
                self.create_summary(tf.summary.scalar, 's1_ord', s1[0])
                
                self.logdensities.append(s1_d.logdens(s1))

                s1_prior = s1_prior_d.logdens(s1)
                self.priors.append(s1_prior)
                
            PWalk = NormalRW(dim=None, sigma0=None, mu0=None, sigma=s1, name='OrdWalk')
            self.PWalk = PWalk
                
    def create_walk_inference(self, mu=None):
        dim = self.dim
        gvar = GVAR(dim=dim[0]*dim[1], len=self.NUM_STEPS, name='coef_rw_inference', mu=mu)
        outputs = gvar.sample()
        
        self.logdensities.append(gvar.logdens)
        self.priors.append(self.PWalk.logdens(outputs, reduce=True))
        self.outputs = outputs
        
        return outputs
    
    def create_observ_dispersion_inference(self, prior_disp):
        with tf.variable_scope('obs_d_inf', reuse=tf.AUTO_REUSE):
#             ldiag = DFlow([NVPFlow(dim=3, name='ldiag_flow_' + str(i)) for i in range(2)], init_sigma=0.05)
            ldiag = DFlow([LinearChol(dim=3, name='ldiag_flow_' + str(i)) for i in range(1)], init_sigma=0.05)

            ldiag.output -= 0.5*math.log(prior_disp)
            ldiag.logdens -= tf.reduce_sum(ldiag.output, axis=-1)
        
        self.obs_d = MVNormal(3, sigma=None, name='obs_d_prior', 
                   ldiag=ldiag.output[0])
        
        df = 3
        pmat = np.diag([(2./prior_disp)]*3)/df
        cov_prior = WishartCholesky(df, pmat, cholesky_input_output_matrices=True)
        
        pr = cov_prior.log_prob(self.obs_d.fsigma)
        self.logdensities.append(ldiag.logdens[0])
        self.priors.append(pr)
        
        sigmas = tf.sqrt(tf.diag_part(self.obs_d.sigma))
        
        std = tf.nn.moments(self.data[0,1:,:3] - self.data[0,:-1,:3], axes=[0])[1]
        print(std, sigmas)
        rsquareds = 1 - sigmas/std
        self.create_summary(tf.summary.scalar, 'rsquared_post_mean', tf.reduce_mean(rsquareds))
        self.create_summary(tf.summary.histogram, 'post_rsquared', rsquareds)
        self.create_summary(tf.summary.histogram, 'post_disp', sigmas)
            
    def predict(self, observable_mask, outputs):
        dim = self.dim
        data = self.data
        out = tf.reshape(outputs, [self.NUM_STEPS, dim[0], dim[1]])

        def step(prev, x):
            mask = x[0]
            prev_pred = tf.where(mask, x[1], prev)[tf.newaxis]
            params = x[2]

            d0 = params[:,:dim[0]]
            d1 = params[:,dim[0]:2*dim[0]]

            pp1 = prev_pred[:,:dim[0]]
            pp0 = prev_pred[:,dim[0]:2*dim[0]]

            new_pred = tf.matmul(pp0, d0)[0] + tf.matmul(pp1, d1)[0]+ params[:,-1] + pp1[0]
            obs_noise = 0.#elf.obs_d.sample()
            new_pred = tf.where(mask, new_pred, new_pred + obs_noise)
            
            new_pred = tf.concat([new_pred, pp1[0]], axis=0)
            return new_pred

        ar = tf.scan(step, [observable_mask, data[0], out], initializer=tf.zeros([2*dim[0]], dtype=floatX))
        return ar
    
    def create_likelihood(self, observable_mask, outputs):
        dim = self.dim
        obs_d = self.obs_d
        
        preds = self.predict(observable_mask, outputs)
        self.preds = preds[:,:3]
        
        diffs = preds[:-1] - self.data[0,1:]
        diffs = diffs[:,:dim[0]]
        
        std = tf.nn.moments(self.data[0,1:,:3] - self.data[0,:-1,:3], axes=[0])[1]
        rsq_obs = tf.reduce_mean(tf.square(diffs), axis=0)/std
        rsq_obs = 1-tf.reduce_mean(rsq_obs)
        self.create_summary(tf.summary.scalar, 'rsquared_observed', rsq_obs)

        logl = obs_d.logdens(diffs, reduce=False)
        logl *= tf.cast(self.observable_mask[1:], floatX)
        
        logl = tf.reduce_sum(logl)
        self.create_summary(tf.summary.scalar, 'loglikelihood', logl)
        self.priors.append(logl)

In [13]:
current_year = tf.placeholder(tf.int32, shape=(), name='current_year')

In [14]:
global_inf = DFlow([NVPFlow(dim=(3*2+1)*3, name='flow_{}'.format(i)) for i in range(6)], init_sigma=0.08)
# global_inf = DFlow([LinearChol(dim=(3*2+1)*3, name='flow_{}'.format(i)) for i in range(1)], init_sigma=0.08)

global_prior = Normal(None, sigma=30.).logdens(global_inf.output)
tf.add_to_collection('priors', global_prior)
tf.add_to_collection('logdensities', global_inf.logdens)

In [15]:
global_inf.logdens

<tf.Tensor 'sub:0' shape=(1,) dtype=float64>

In [16]:
global_inf.output

<tf.Tensor 'flow_5/add_3:0' shape=(1, 21) dtype=float64>

In [17]:
with tf.variable_scope('variation_rate', dtype=floatX):
#     with tf.variable_scope('prior'):
#         vprior = tf.distributions.Exponential(rate=1/0.7)
        
    variation_prerate = tf.get_variable('prerate', initializer=math.log(0.7))
    variation_rate = tf.exp(variation_prerate)
#     variation_d = tf.distributions.Exponential(variation_rate)
    variation = variation_rate#variation_d.sample()
    
#     lpd = tf.cast(variation_d.log_prob(variation), floatX)[tf.newaxis]
#     lpp = tf.cast(vprior.log_prob(variation), floatX)
    variation = tf.cast(variation, floatX)
    
    tf.summary.scalar('variation', variation)
#     tf.add_to_collection('logdensities', lpd)
#     tf.add_to_collection('priors', lpp)

In [18]:
variation

<tf.Tensor 'variation_rate/Cast:0' shape=() dtype=float64>

In [19]:
individ_variation_prior = Normal((3*2+1)*3, sigma=variation, mu=global_inf.output[0])

In [20]:
models = []
indiv_logdens = []
indiv_priors = []
indivs = {}

with tf.variable_scope(tf.get_variable_scope(), dtype=floatX, reuse=tf.AUTO_REUSE):
    for country, data in country_data.items():
        with tf.variable_scope(country):
#             individ_variation = DFlow([LinearChol((3*2+1)*3, name='lc')], init_sigma=0.08)
            individ_variation = DFlow([NVPFlow((3*2+1)*3, 
                                               name='nvp_{}'.format(i), 
                                               aux_vars=global_inf.output) for i in range(6)], init_sigma=0.08)

            ind = individ_variation.output[0]
            indivs[country] = ind

            indiv_logdens.append(individ_variation.logdens)
            indiv_priors.append(individ_variation_prior.logdens(ind))

        model = VARmodel(data, name='{}_model'.format(country), mu=ind[tf.newaxis], current_year=current_year)
        models.append(model)

Tensor("AUS_model/coef_rw_inference_1/VAR/add:0", shape=(35, 21), dtype=float64)
Tensor("AUS_model/moments/Squeeze_1:0", shape=(3,), dtype=float64) Tensor("AUS_model/Sqrt:0", shape=(3,), dtype=float64)
Tensor("FRA_model/coef_rw_inference_1/VAR/add:0", shape=(32, 21), dtype=float64)
Tensor("FRA_model/moments/Squeeze_1:0", shape=(3,), dtype=float64) Tensor("FRA_model/Sqrt:0", shape=(3,), dtype=float64)
Tensor("GBR_model/coef_rw_inference_1/VAR/add:0", shape=(23, 21), dtype=float64)
Tensor("GBR_model/moments/Squeeze_1:0", shape=(3,), dtype=float64) Tensor("GBR_model/Sqrt:0", shape=(3,), dtype=float64)


In [21]:
indiv_logdens

[<tf.Tensor 'AUS/sub:0' shape=(1,) dtype=float64>,
 <tf.Tensor 'FRA/sub:0' shape=(1,) dtype=float64>,
 <tf.Tensor 'GBR/sub:0' shape=(1,) dtype=float64>]

In [22]:
graph = tf.get_default_graph()
prior = tf.reduce_sum([model.priors for model in models])\
+ tf.reduce_sum(indiv_priors) + tf.reduce_sum(graph.get_collection('priors'))

logdensity = tf.reduce_sum([model.logdensities for model in models])\
+ tf.reduce_sum(indiv_logdens) + tf.reduce_sum(graph.get_collection('logdensities'))

In [23]:
kl = logdensity - prior
kl /= 36*21

In [24]:
kl

<tf.Tensor 'truediv:0' shape=() dtype=float64>

In [25]:
kls = tf.summary.scalar('KLd', kl)
summary = tf.summary.merge_all()#tf.summary.merge([kls, tf.summary.scalar('prior', prior)] + [model.summaries for model in models])

In [26]:
main_op = tf.train.AdamOptimizer(0.0001).minimize(kl)

In [27]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [28]:
init.run()

In [29]:
# !rm -R /home/nikita/tmp/hier

In [30]:
writer = tf.summary.FileWriter('/home/nikita/tmp/hier/4main++')

In [31]:
# writer.add_graph(tf.get_default_graph())

In [32]:
models[0].preds

<tf.Tensor 'AUS_model/strided_slice_5:0' shape=(35, 3) dtype=float64>

In [33]:
epoch = 0

In [34]:
def validate_year(year):
    cdic = {model.name:model for model in models}
    preds = {model.name:[] for model in models}
    preds_t = {model.name: model.preds for model in models}
    for step in range(1000):
        preds_i = sess.run(preds_t, {current_year:year})
        for k in preds.keys():
            preds[k].append(preds_i[k][cdic[k].years > year])
            
    mean_pred = {k:np.mean(v, axis=0) for k,v in preds.items()}
    current_vals = {model.name:model.data_raw.loc[:,year].values[:3]}
    return mean_pred, current_vals

In [35]:
validate_year(1990)

({'AUS_model': array([[  6.9902475 ,  10.47738941,   7.21347428],
         [  6.93888002,  11.08745949,   7.52077839],
         [  6.96557878,  11.83085523,   7.96083573],
         [  7.07702135,  12.75291006,   8.55601154],
         [  7.262492  ,  13.88035811,   9.33242984],
         [  7.5331935 ,  15.23308218,  10.3364206 ],
         [  7.91036133,  16.86330209,  11.62256169],
         [  8.43980527,  18.8413549 ,  13.26440228],
         [  9.14046311,  21.2670231 ,  15.36957294],
         [ 10.06142082,  24.29336475,  18.03866059],
         [ 11.22666359,  28.06656522,  21.41920566],
         [ 12.72266063,  32.78151862,  25.70654585],
         [ 14.71397048,  38.64177467,  31.30106224],
         [ 17.30345409,  46.18543574,  38.56031085],
         [ 20.64598938,  55.96543134,  47.99851556],
         [ 24.98711782,  68.71603535,  60.09733366],
         [ 30.52505609,  85.38007162,  75.93418074],
         [ 37.87948934, 106.92020655,  96.69155145],
         [ 47.06305988, 134.88362

In [ ]:
for epoch in range(epoch, 100000):
    fd = {current_year:3000}
    for step in range(100):
        sess.run(main_op, fd)
    s, _ = sess.run([summary, main_op], fd)
    writer.add_summary(s, global_step=epoch)

In [ ]:
indivs

In [ ]:
global_post=global_inf.output[0]

In [ ]:
ss = []
for _ in range(10000):
    ss.append(indivs['FRA'].eval())
ss = np.array(ss)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
ss.std(axis=0)

In [ ]:
sns.kdeplot(ss[:,1], ss[:,2])
plt.show()

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.restore(sess,'/tmp/save')

In [ ]:
epoch

In [ ]:
epoch = 1065